In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
from nb_007b import *

# IMDB

## Fine-tuning the LM

Data has been prepared in csv files at the beginning 007a, we will use it know.

### Loading the data

In [ ]:
PATH = Path('../data/aclImdb/')
CLAS_PATH = PATH/'clas'
LM_PATH = PATH/'lm'
MODEL_PATH = PATH/'models'
os.makedirs(CLAS_PATH, exist_ok=True)
os.makedirs(LM_PATH, exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)

In [ ]:
data = data_from_textcsv(LM_PATH, Tokenizer(), data_func=lm_data, bs=50)
learn = RNNLearner.language_model(data, drop_mult=0.3, pretrained_fnames=['lstm_wt103', 'itos_wt103'])
learn.freeze()

In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

Total time: 15:38
epoch  train loss  valid loss  accuracy
0      4.579638    4.458626    0.254510  (15:38)



In [ ]:
learn.save('fit_head')

In [ ]:
learn.load('fit_head')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7), pct_start=0.25)

Total time: 2:54:40
epoch  train loss  valid loss  accuracy
0      4.337709    4.245524    0.276793  (17:27)
1      4.239955    4.163144    0.287510  (17:23)
2      4.140783    4.114172    0.293098  (17:24)
3      4.086738    4.080914    0.297168  (17:19)
4      4.020078    4.050975    0.300374  (17:21)
5      3.969319    4.026036    0.303098  (17:23)
6      3.920115    4.004704    0.305691  (17:31)
7      3.922437    3.991960    0.307355  (17:38)
8      3.820791    3.981143    0.308614  (17:31)
9      3.830508    3.978769    0.308737  (17:37)



In [ ]:
learn.save('fine_tuned')

In [ ]:
learn.save_encoder('fine_tuned_enc')

## Classifier

We need to use the same vocab as for the LM.

In [ ]:
shutil.copy(LM_PATH/'models'/'fine_tuned_enc.pth', CLAS_PATH/'models'/'fine_tuned_enc.pth')
shutil.copy(LM_PATH/'tmp'/'itos.pkl', CLAS_PATH/'tmp'/'itos.pkl')

PosixPath('../data/aclImdb/clas/tmp/itos.pkl')

In [ ]:
data = data_from_textcsv(CLAS_PATH, Tokenizer(), vocab=Vocab(LM_PATH/'tmp'), data_func=classifier_data, bs=50)
learn = RNNLearner.classifier(data, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc')
learn.freeze()

Numericalizing train.
Numericalizing valid.


In [ ]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

Total time: 02:22
epoch  train loss  valid loss  accuracy
0      0.287135    0.190471    0.927440  (02:22)



In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/2.6,1e-2), moms=(0.8,0.7), pct_start=0.1)

Total time: 02:43
epoch  train loss  valid loss  accuracy
0      0.231930    0.157012    0.942080  (02:43)



In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-2/(2.6)**4,1e-2), moms=(0.8,0.7), pct_start=0.1)

Total time: 10:34
epoch  train loss  valid loss  accuracy
0      0.202586    0.172194    0.935240  (05:17)
1      0.105323    0.154032    0.946560  (05:17)



In [ ]:
learn.load('first')

In [ ]:
lr=1e-2
lrm = 2.6
lrs = np.array([lr/(lrm**4), lr/(lrm**3), lr/(lrm**2), lr/lrm, lr])

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, lrs, moms=(0.8,0.7), pct_start=0.1)

Total time: 02:43
epoch  train loss  valid loss  accuracy
0      0.264912    0.167870    0.937400  (02:43)



In [ ]:
learn.save('second')

In [ ]:
learn.load('second')

In [ ]:
lr=5e-3
lrm = 2.6
lrs = np.array([lr/(lrm**4), lr/(lrm**3), lr/(lrm**2), lr/lrm, lr])

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, lrs, moms=(0.8,0.7), pct_start=0.1)

Total time: 04:03
epoch  train loss  valid loss  accuracy
0      0.223868    0.151519    0.945080  (04:03)



In [ ]:
learn.save('third')

In [ ]:
learn.load('third')

In [ ]:
lr=1e-3
lrm = 2.6
lrs = np.array([lr/(lrm**4), lr/(lrm**3), lr/(lrm**2), lr/lrm, lr])

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, lrs, moms=(0.8,0.7), pct_start=0.1)

Total time: 10:31
epoch  train loss  valid loss  accuracy
0      0.133141    0.167304    0.946960  (05:15)
1      0.127248    0.171328    0.947680  (05:16)

